# Capstone Project
#### That notebook is a part of Applied Data Science Capstone project.
By Andrei Staradubets

## Part 1

In [21]:
import numpy as np
import pandas as pd
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Part 2

In [22]:
from bs4 import BeautifulSoup
import requests

Let's get table from the wiki page...

In [76]:
# getting data from internet
URL='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
raw_page= requests.get(URL).text

# using beautiful soup to parse the HTML/XML codes.
soup = BeautifulSoup(raw_page,'xml')
table = soup.find('table') #table on the page

... and proccess it into row dataframe

In [94]:
Postcode      = []
Borough       = []
Neighbourhood = []
for tr_cell in table.find_all('tr'): #table rows
    counter = 0
    td_cells=tr_cell.find_all('td') #table sells
    try: #cuz first row doesn't have td cells
        Postcode_var      = td_cells[0].text
        Borough_var       = td_cells[1].text
        Neighbourhood_var = str(td_cells[2].text).strip()
        
        if (Postcode_var == 'Not assigned' or Borough_var == 'Not assigned'):
            continue
        if(Neighbourhood_var == 'Not assigned'):
            Neighbourhood_var=Borough_var
            
        Postcode.append(Postcode_var)
        Borough.append(Borough_var)
        Neighbourhood.append(Neighbourhood_var)
    except:
        pass
d = {'Postcode':Postcode,'Borough':Borough,'Neighbourhood':Neighbourhood}
df=pd.DataFrame(d)

Row DataFrame

In [95]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


Let's group by postcode (also with Borough) and concat neighbourhoods in that postcodes

In [89]:
dftest=df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ', '.join(x)).reset_index()

In [91]:
dftest.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Shape is...

In [96]:
print (dftest.shape)

(103, 3)
